#### Global values

In [1]:
IMAGE_PATH = "./images/ncr_8338d_mcmaster_mz_mit20x2.jpg"       
ROI = (2112, 2400, 7600, 5740)  # <-- EDIT to your test region

# STEP 1
#### -Look for vertical vias

In [2]:
# CHUNK 1R (Rectangular 30x40): ROI scan with ZNCC/SAD, IoU-free NMS (morph-based), and required overlays.
# - One orientation ONLY: width=30, height=40 (rotate later).
# - Outputs: *_overlay_boxes.png, *_overlay_heatmap.png, *_overlay_both.png, *_detections.csv, *_params.json (ROI only).
# - Edit CONFIG paths and ROI before running.

import os
import json
import math
import csv
from datetime import datetime

import numpy as np
import cv2

# =========================
# CONFIG (edit these)
# =========================
# IMAGE_PATH = "./images/ncr_8338d_mcmaster_mz_mit20x2.jpg"                  # Original image (grayscale or color)
TEMPLATE_PATHS = [
    "./input-metal-via-contacts/contact_v001.png",
    "./input-metal-via-contacts/contact_v003.png",
    "./input-metal-via-contacts/contact_v003.png",
    "./input-metal-via-contacts/contact_v004.png",
    "./input-metal-via-contacts/contact_v005.png",
    "./input-metal-via-contacts/contact_v006.png",
    "./input-metal-via-contacts/contact_v007.png",
    "./input-metal-via-contacts/contact_v008.png",
    "./input-metal-via-contacts/contact_v009.png",
    "./input-metal-via-contacts/contact_v010.png"
    # "PATH/TO/contact_30x40_example2.png",
]
OUTPUT_PREFIX = "./output-metal-via-contacts/mioc_roi_rect_V"        # Prefix for output files
METRIC = "ZNCC"                                       # "ZNCC" (recommended) or "SAD"
TEMPLATE_W, TEMPLATE_H = 30, 40                       # <-- One orientation (w=30, h=40). We'll rotate later if needed.
NMS_RADIUS = 12                                       # ~0.4 * min(w,h) ≈ 12 for 30x40
THRESH_ZNCC = 0.65                                    # Initial ZNCC threshold
THRESH_SAD_INV = 0.80                                 # If METRIC="SAD": use 1 - normalized-SAD >= this
DRAW_LABELS = True                                    # Score labels next to boxes

# ROI: (x0, y0, w, h) — keep manageable for calibration; expand later
# ROI = (2112, 2400, 7600, 5740)  # <-- EDIT to your test region

# =========================
# UTILITIES
# =========================
def load_gray_image(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"Failed to read: {path}")
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def load_templates_rect(paths, tw, th):
    tmpls = []
    for p in paths:
        t = cv2.imread(p, cv2.IMREAD_UNCHANGED)
        if t is None:
            raise FileNotFoundError(f"Template not found: {p}")
        if t.ndim == 3:
            t = cv2.cvtColor(t, cv2.COLOR_BGR2GRAY)
        if t.shape[::-1] != (tw, th):  # shape[::-1] gives (width,height)
            t = cv2.resize(t, (tw, th), interpolation=cv2.INTER_AREA)
        t = t.astype(np.float32, copy=False)
        tmpls.append(t)
    if not tmpls:
        raise ValueError("No templates loaded.")
    return tmpls

def method_for(metric):
    metric = metric.upper()
    if metric == "ZNCC":
        return cv2.TM_CCOEFF_NORMED
    elif metric == "SAD":
        return cv2.TM_SQDIFF_NORMED
    else:
        raise ValueError("METRIC must be 'ZNCC' or 'SAD'")

def compute_score_maps(roi_img, templates, metric):
    """
    Returns:
        max_score_map (H', W') float32, higher is better
        best_id_map   (H', W') int16, index of template giving max score at each position
        raw_maps      list of per-template raw maps BEFORE any inversion
    """
    m = method_for(metric)
    roi32 = roi_img.astype(np.float32, copy=False)
    raw_maps = []
    for t in templates:
        r = cv2.matchTemplate(roi32, t, m)  # result size: (h - th + 1, w - tw + 1)
        raw_maps.append(r.astype(np.float32, copy=False))
    raw_stack = np.stack(raw_maps, axis=-1)  # (H', W', n_templates)

    if metric.upper() == "ZNCC":
        max_score_map = raw_stack.max(axis=-1)
        best_id_map = raw_stack.argmax(axis=-1).astype(np.int16)
    else:
        inv_stack = 1.0 - raw_stack
        max_score_map = inv_stack.max(axis=-1)
        best_id_map = inv_stack.argmax(axis=-1).astype(np.int16)

    return max_score_map, best_id_map, raw_maps

def nms_points(score_map, threshold, radius):
    """
    Morphological NMS. Returns list of (y, x, score) in score_map coords for local maxima above threshold.
    """
    H, W = score_map.shape
    r = max(1, int(radius))
    k = 2 * r + 1
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (k, k))
    local_max = cv2.dilate(score_map, kernel)
    maxima_mask = (score_map >= local_max - 1e-12) & (score_map >= threshold)
    ys, xs = np.where(maxima_mask)
    pts = [(int(y), int(x), float(score_map[y, x])) for (y, x) in zip(ys, xs)]
    pts.sort(key=lambda a: -a[2])
    return pts

def clip_box(tlx, tly, brx, bry, W, H):
    return max(0, tlx), max(0, tly), min(W - 1, brx), min(H - 1, bry)

def draw_boxes_rect(image_bgr, centers_xy, w_box, h_box, color=(0, 0, 255), thickness=2, draw_labels=False):
    """
    centers_xy: list of (x, y, score) in full-image coords (center points)
    w_box, h_box: rectangle size
    """
    out = image_bgr.copy()
    H, W = out.shape[:2]
    half_w = w_box // 2
    half_h = h_box // 2
    for (cx, cy, s) in centers_xy:
        tlx, tly = int(round(cx - half_w)), int(round(cy - half_h))
        brx, bry = tlx + w_box - 1, tly + h_box - 1
        tlx, tly, brx, bry = clip_box(tlx, tly, brx, bry, W, H)
        cv2.rectangle(out, (tlx, tly), (brx, bry), color, thickness, lineType=cv2.LINE_AA)
        # Crosshair
        ch = max(3, min(w_box, h_box) // 6)
        cv2.line(out, (cx - ch, cy), (cx + ch, cy), color, 1, lineType=cv2.LINE_AA)
        cv2.line(out, (cx, cy - ch), (cx, cy + ch), color, 1, cv2.LINE_AA)
        if draw_labels:
            txt = f"{s:.2f}"
            cv2.putText(out, txt, (tlx, max(0, tly - 2)), cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1, cv2.LINE_AA)
    return out

def make_heatmap_overlay_aniso(base_gray_bgr, roi_rect, center_scores_map, alpha=0.40, kx=7, ky=11):
    """
    Anisotropic smoothing for rectangular templates. kx,ky must be odd.
    center_scores_map: H_roi x W_roi
    """
    x0, y0, w, h = roi_rect
    cm = center_scores_map.copy().astype(np.float32)
    if cm.max() > cm.min():
        cm = (cm - cm.min()) / (cm.max() - cm.min())
    cm_uint8 = np.clip(cm * 255.0, 0, 255).astype(np.uint8)
    color = cv2.applyColorMap(cm_uint8, cv2.COLORMAP_JET)
    overlay = base_gray_bgr.copy()
    roi_base = overlay[y0:y0+h, x0:x0+w]
    blended = cv2.addWeighted(color, alpha, roi_base, 1 - alpha, 0.0)
    overlay[y0:y0+h, x0:x0+w] = blended
    return overlay

def centers_from_scoremap_rect(points_yx, roi_rect, tw, th, best_id_map):
    """
    Convert NMS points (y, x, score) in score_map coords to full-image centers & template IDs.
    score_map coords correspond to TOP-LEFT placement of the template window (tw x th).
    """
    x0, y0, _, _ = roi_rect
    half_w = tw // 2
    half_h = th // 2
    out = []
    for (y, x, s) in points_yx:
        cx = x + half_w + x0
        cy = y + half_h + y0
        tid = int(best_id_map[y, x])
        out.append({"x": int(cx), "y": int(cy), "score": float(s), "template_id": tid, "width": tw, "height": th})
    return out

def write_csv(path, detections):
    with open(path, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=["x", "y", "score", "template_id", "width", "height", "band_id"])
        w.writeheader()
        for d in detections:
            row = dict(d)
            row["band_id"] = 0  # ROI-only run uses band 0
            w.writerow(row)

# =========================
# RUN (ROI only)
# =========================
# Load image
img_gray = load_gray_image(IMAGE_PATH)
H_full, W_full = img_gray.shape[:2]
img_bgr = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

# Load rectangular templates (single orientation)
templates = load_templates_rect(TEMPLATE_PATHS, TEMPLATE_W, TEMPLATE_H)

# ROI crop
x0, y0, w, h = ROI
assert 0 <= x0 < W_full and 0 <= y0 < H_full, "ROI origin out of image"
assert x0 + w <= W_full and y0 + h <= H_full, "ROI extends beyond image"
roi = img_gray[y0:y0+h, x0:x0+w]

# Compute score maps across templates
max_score_map, best_id_map, _raw = compute_score_maps(roi, templates, METRIC)

# Threshold selection
thr = float(THRESH_ZNCC) if METRIC.upper() == "ZNCC" else float(THRESH_SAD_INV)

# NMS in score-map space (size: (h - TEMPLATE_H + 1, w - TEMPLATE_W + 1))
points_yx = nms_points(max_score_map, threshold=thr, radius=NMS_RADIUS)

# Build detections (full-image centers)
detections = centers_from_scoremap_rect(points_yx, ROI, TEMPLATE_W, TEMPLATE_H, best_id_map)

# Build a center-score map at ROI resolution for heatmap (place scores at centers)
center_map = np.zeros((h, w), dtype=np.float32)
half_w = TEMPLATE_W // 2
half_h = TEMPLATE_H // 2
for (y, x, s) in points_yx:
    cy = y + half_h
    cx = x + half_w
    if 0 <= cy < h and 0 <= cx < w:
        center_map[cy, cx] = max(center_map[cy, cx], s)

# Anisotropic smoothing tuned to rectangle size
# Ensure odd kernel sizes
kx = max(3, (TEMPLATE_W // 2) * 2 + 1)
ky = max(3, (TEMPLATE_H // 2) * 2 + 1)
center_map_blur = cv2.GaussianBlur(center_map, (kx, ky), 0)

# Overlays
overlay_heat = make_heatmap_overlay_aniso(img_bgr, ROI, center_map_blur, alpha=0.42, kx=kx, ky=ky)

# Draw boxes on original and on heatmap (true rectangle size)
centers_xy = [(d["x"], d["y"], d["score"]) for d in detections]
overlay_boxes = draw_boxes_rect(img_bgr, centers_xy, TEMPLATE_W, TEMPLATE_H, color=(0, 0, 255), thickness=2, draw_labels=DRAW_LABELS)
overlay_both = draw_boxes_rect(overlay_heat, centers_xy, TEMPLATE_W, TEMPLATE_H, color=(0, 0, 255), thickness=2, draw_labels=DRAW_LABELS)

# =========================
# SAVE ARTIFACTS
# =========================
boxes_path = f"{OUTPUT_PREFIX}_overlay_boxes.png"
heat_path = f"{OUTPUT_PREFIX}_overlay_heatmap.png"
both_path = f"{OUTPUT_PREFIX}_overlay_both.png"
csv_path = f"{OUTPUT_PREFIX}_detections.csv"
params_path = f"{OUTPUT_PREFIX}_params.json"

cv2.imwrite(boxes_path, overlay_boxes)
cv2.imwrite(heat_path, overlay_heat)
cv2.imwrite(both_path, overlay_both)
write_csv(csv_path, detections)

params = {
    "timestamp": datetime.now().isoformat(),
    "image_path": os.path.abspath(IMAGE_PATH),
    "template_paths": [os.path.abspath(p) for p in TEMPLATE_PATHS],
    "metric": METRIC,
    "template_size": {"width": TEMPLATE_W, "height": TEMPLATE_H},
    "nms_radius": NMS_RADIUS,
    "threshold_used": thr,
    "roi": {"x0": x0, "y0": y0, "w": w, "h": h},
    "notes": "ROI-only rectangular-template run (one orientation). Full-image banding + checkpointing next."
}
with open(params_path, "w") as f:
    json.dump(params, f, indent=2)

# =========================
# SUMMARY
# =========================
print(f"[OK] ROI size: {w}x{h} @ ({x0},{y0})")
print(f"[OK] Template size: {TEMPLATE_W}x{TEMPLATE_H} (w x h)")
print(f"[OK] Score-map size: {max_score_map.shape[1]}x{max_score_map.shape[0]}")
print(f"[OK] Detections: {len(detections)} (metric={METRIC}, threshold={thr})")
print(f"[OUT] {boxes_path}, {heat_path}, {both_path}, {csv_path}, {params_path}")
if len(detections) > 0:
    print("Top 5 detections:", detections[:5])
else:
    print("No detections at current threshold. Consider lowering THRESH_ZNCC (e.g., 0.65) or verifying template quality/alignment.")


[OK] ROI size: 7600x5740 @ (2112,2400)
[OK] Template size: 30x40 (w x h)
[OK] Score-map size: 7571x5701
[OK] Detections: 409 (metric=ZNCC, threshold=0.65)
[OUT] ./output-metal-via-contacts/mioc_roi_rect_V_overlay_boxes.png, ./output-metal-via-contacts/mioc_roi_rect_V_overlay_heatmap.png, ./output-metal-via-contacts/mioc_roi_rect_V_overlay_both.png, ./output-metal-via-contacts/mioc_roi_rect_V_detections.csv, ./output-metal-via-contacts/mioc_roi_rect_V_params.json
Top 5 detections: [{'x': 6613, 'y': 5433, 'score': 0.9999943375587463, 'template_id': 0, 'width': 30, 'height': 40}, {'x': 3280, 'y': 5464, 'score': 0.999991238117218, 'template_id': 5, 'width': 30, 'height': 40}, {'x': 5585, 'y': 5590, 'score': 0.9999908804893494, 'template_id': 3, 'width': 30, 'height': 40}, {'x': 7595, 'y': 4684, 'score': 0.9999901652336121, 'template_id': 1, 'width': 30, 'height': 40}, {'x': 4818, 'y': 6637, 'score': 0.9999895095825195, 'template_id': 4, 'width': 30, 'height': 40}]


# STEP 2
#### -Look for horizontal vias

In [3]:
# CHUNK 1H (Horizontal 40x30): ROI scan with ZNCC/SAD using YOUR horizontal templates (no rotation),
# with overlays (green boxes), heatmap, combined overlay, CSV, and params JSON.
# Edit CONFIG paths and ROI before running.

import os
import json
import math
import csv
from datetime import datetime

import numpy as np
import cv2

# =========================
# CONFIG (edit these)
# =========================
# IMAGE_PATH = "./images/ncr_8338d_mcmaster_mz_mit20x2.jpg"                  # Original image (grayscale or color)
TEMPLATE_PATHS = [
    "./input-metal-via-contacts/contact_h001.png",
    "./input-metal-via-contacts/contact_h003.png",
    "./input-metal-via-contacts/contact_h003.png",
    "./input-metal-via-contacts/contact_h004.png",
    "./input-metal-via-contacts/contact_h005.png",
    "./input-metal-via-contacts/contact_h006.png",
    "./input-metal-via-contacts/contact_h007.png",
    "./input-metal-via-contacts/contact_h008.png",
    "./input-metal-via-contacts/contact_h009.png"
    # "PATH/TO/contact_30x40_example2.png",
]
OUTPUT_PREFIX = "./output-metal-via-contacts/mioc_roi_rect_H"        # Prefix for output files
METRIC = "ZNCC"                                       # "ZNCC" (recommended) or "SAD"
TEMPLATE_W, TEMPLATE_H = 40, 30                       # <-- One orientation (w=30, h=40). We'll rotate later if needed.
NMS_RADIUS = 12                                       # ~0.4 * min(w,h) ≈ 12 for 30x40
THRESH_ZNCC = 0.65                                    # Initial ZNCC threshold
THRESH_SAD_INV = 0.80                                 # If METRIC="SAD": use 1 - normalized-SAD >= this
DRAW_LABELS = True                                    # Score labels next to boxes

# ROI: (x0, y0, w, h) — keep manageable for calibration; expand later
# ROI = (2112, 2400, 7600, 5400)  # <-- EDIT to your test region

# =========================
# UTILITIES
# =========================
def load_gray_image(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"Failed to read: {path}")
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def load_templates_rect(paths, tw, th):
    """Load templates; enforce exact (th, tw) by resizing if needed. No rotation."""
    tmpls = []
    for p in paths:
        t = cv2.imread(p, cv2.IMREAD_UNCHANGED)
        if t is None:
            raise FileNotFoundError(f"Template not found: {p}")
        if t.ndim == 3:
            t = cv2.cvtColor(t, cv2.COLOR_BGR2GRAY)
        if t.shape != (th, tw):  # shape is (H, W)
            t = cv2.resize(t, (tw, th), interpolation=cv2.INTER_AREA)
        tmpls.append(t.astype(np.float32, copy=False))
    if not tmpls:
        raise ValueError("No templates loaded.")
    return tmpls

def method_for(metric):
    m = metric.upper()
    if m == "ZNCC": return cv2.TM_CCOEFF_NORMED
    if m == "SAD":  return cv2.TM_SQDIFF_NORMED
    raise ValueError("METRIC must be 'ZNCC' or 'SAD'")

def compute_score_maps(roi_img, templates, metric):
    """
    Returns:
        max_score_map (H', W') float32, higher is better
        best_id_map   (H', W') int16, index of template giving max score at each position
        raw_maps      list of per-template raw maps BEFORE any inversion
    """
    m = method_for(metric)
    roi32 = roi_img.astype(np.float32, copy=False)
    raw_maps = []
    for t in templates:
        r = cv2.matchTemplate(roi32, t, m)  # result: (h - th + 1, w - tw + 1)
        raw_maps.append(r.astype(np.float32, copy=False))
    raw_stack = np.stack(raw_maps, axis=-1)  # (H', W', n_templates)
    if metric.upper() == "ZNCC":
        max_score_map = raw_stack.max(axis=-1)
        best_id_map   = raw_stack.argmax(axis=-1).astype(np.int16)
    else:
        inv_stack     = 1.0 - raw_stack
        max_score_map = inv_stack.max(axis=-1)
        best_id_map   = inv_stack.argmax(axis=-1).astype(np.int16)
    return max_score_map, best_id_map, raw_maps

def nms_points(score_map, threshold, radius):
    """Morphological NMS. Returns list of (y, x, score) in score_map coords for local maxima above threshold."""
    r = max(1, int(radius))
    k = 2 * r + 1
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (k, k))
    local_max = cv2.dilate(score_map, kernel)
    maxima_mask = (score_map >= local_max - 1e-12) & (score_map >= threshold)
    ys, xs = np.where(maxima_mask)
    pts = [(int(y), int(x), float(score_map[y, x])) for (y, x) in zip(ys, xs)]
    pts.sort(key=lambda a: -a[2])
    return pts

def clip_box(tlx, tly, brx, bry, W, H):
    return max(0, tlx), max(0, tly), min(W - 1, brx), min(H - 1, bry)

def draw_boxes_rect(image_bgr, centers_xy, w_box, h_box, color=(0, 255, 0), thickness=2, draw_labels=False):
    """Green boxes for horizontal orientation."""
    out = image_bgr.copy()
    H, W = out.shape[:2]
    half_w = w_box // 2
    half_h = h_box // 2
    for (cx, cy, s) in centers_xy:
        tlx, tly = int(round(cx - half_w)), int(round(cy - half_h))
        brx, bry = tlx + w_box - 1, tly + h_box - 1
        tlx, tly, brx, bry = clip_box(tlx, tly, brx, bry, W, H)
        cv2.rectangle(out, (tlx, tly), (brx, bry), color, thickness, lineType=cv2.LINE_AA)
        ch = max(3, min(w_box, h_box) // 6)  # crosshair
        cv2.line(out, (cx - ch, cy), (cx + ch, cy), color, 1, lineType=cv2.LINE_AA)
        cv2.line(out, (cx, cy - ch), (cx, cy + ch), color, 1, lineType=cv2.LINE_AA)
        if draw_labels:
            cv2.putText(out, f"{s:.2f}", (tlx, max(0, tly - 2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1, cv2.LINE_AA)
    return out

def make_heatmap_overlay_aniso(base_gray_bgr, roi_rect, center_scores_map, alpha=0.40, kx=7, ky=7):
    """Blend a smoothed center-score map (anisotropic kernel) over the ROI."""
    x0, y0, w, h = roi_rect
    cm = center_scores_map.copy().astype(np.float32)
    if cm.max() > cm.min():
        cm = (cm - cm.min()) / (cm.max() - cm.min())
    cm_uint8 = np.clip(cm * 255.0, 0, 255).astype(np.uint8)
    color = cv2.applyColorMap(cm_uint8, cv2.COLORMAP_JET)
    overlay = base_gray_bgr.copy()
    roi_base = overlay[y0:y0+h, x0:x0+w]
    blended = cv2.addWeighted(color, alpha, roi_base, 1 - alpha, 0.0)
    overlay[y0:y0+h, x0:x0+w] = blended
    return overlay

def centers_from_scoremap_rect(points_yx, roi_rect, tw, th, best_id_map):
    """Convert NMS points to full-image centers with template IDs and sizes."""
    x0, y0, _, _ = roi_rect
    half_w = tw // 2
    half_h = th // 2
    out = []
    for (y, x, s) in points_yx:
        cx = x + half_w + x0
        cy = y + half_h + y0
        tid = int(best_id_map[y, x])
        out.append({"x": int(cx), "y": int(cy), "score": float(s), "template_id": tid, "width": tw, "height": th})
    return out

def write_csv(path, detections):
    with open(path, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=["x", "y", "score", "template_id", "width", "height", "band_id"])
        w.writeheader()
        for d in detections:
            row = dict(d)
            row["band_id"] = 0  # ROI-only run uses band 0
            w.writerow(row)

# =========================
# RUN (ROI only)
# =========================
# Load image
img_gray = load_gray_image(IMAGE_PATH)
H_full, W_full = img_gray.shape[:2]
img_bgr = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

# Load horizontal templates (no rotation; resize to 40x30 if needed)
templates = load_templates_rect(TEMPLATE_PATHS, TEMPLATE_W, TEMPLATE_H)

# ROI crop
x0, y0, w, h = ROI
assert 0 <= x0 < W_full and 0 <= y0 < H_full, "ROI origin out of image"
assert x0 + w <= W_full and y0 + h <= H_full, "ROI extends beyond image"
roi = img_gray[y0:y0+h, x0:x0+w]

# Compute score maps across templates
max_score_map, best_id_map, _raw = compute_score_maps(roi, templates, METRIC)

# Threshold selection
thr = float(THRESH_ZNCC) if METRIC.upper() == "ZNCC" else float(THRESH_SAD_INV)

# NMS in score-map space (size: (h - TEMPLATE_H + 1, w - TEMPLATE_W + 1))
points_yx = nms_points(max_score_map, threshold=thr, radius=NMS_RADIUS)

# Build detections (full-image centers)
detections = centers_from_scoremap_rect(points_yx, ROI, TEMPLATE_W, TEMPLATE_H, best_id_map)

# Build a center-score map at ROI resolution for heatmap (place scores at centers)
center_map = np.zeros((h, w), dtype=np.float32)
half_w = TEMPLATE_W // 2
half_h = TEMPLATE_H // 2
for (y, x, s) in points_yx:
    cy = y + half_h
    cx = x + half_w
    if 0 <= cy < h and 0 <= cx < w:
        center_map[cy, cx] = max(center_map[cy, cx], s)

# Anisotropic smoothing tuned to rectangle size
kx = max(3, (TEMPLATE_W // 2) * 2 + 1)  # odd
ky = max(3, (TEMPLATE_H // 2) * 2 + 1)  # odd
center_map_blur = cv2.GaussianBlur(center_map, (kx, ky), 0)

# Overlays (GREEN boxes for horizontal)
overlay_heat  = make_heatmap_overlay_aniso(img_bgr, ROI, center_map_blur, alpha=0.42, kx=kx, ky=ky)
centers_xy    = [(d["x"], d["y"], d["score"]) for d in detections]
overlay_boxes = draw_boxes_rect(img_bgr, centers_xy, TEMPLATE_W, TEMPLATE_H, color=(0, 255, 0), thickness=2, draw_labels=DRAW_LABELS)
overlay_both  = draw_boxes_rect(overlay_heat, centers_xy, TEMPLATE_W, TEMPLATE_H, color=(0, 255, 0), thickness=2, draw_labels=DRAW_LABELS)

# =========================
# SAVE ARTIFACTS
# =========================
boxes_path  = f"{OUTPUT_PREFIX}_overlay_boxes.png"
heat_path   = f"{OUTPUT_PREFIX}_overlay_heatmap.png"
both_path   = f"{OUTPUT_PREFIX}_overlay_both.png"
csv_path    = f"{OUTPUT_PREFIX}_detections.csv"
params_path = f"{OUTPUT_PREFIX}_params.json"

cv2.imwrite(boxes_path, overlay_boxes)
cv2.imwrite(heat_path, overlay_heat)
cv2.imwrite(both_path, overlay_both)
write_csv(csv_path, detections)

params = {
    "timestamp": datetime.now().isoformat(),
    "image_path": os.path.abspath(IMAGE_PATH),
    "template_paths": [os.path.abspath(p) for p in TEMPLATE_PATHS],
    "metric": METRIC,
    "template_size": {"width": TEMPLATE_W, "height": TEMPLATE_H},
    "nms_radius": NMS_RADIUS,
    "threshold_used": thr,
    "roi": {"x0": x0, "y0": y0, "w": w, "h": h},
    "notes": "ROI-only horizontal-template run (40x30) with your provided exemplars. Will merge with vertical in Chunk 3."
}
with open(params_path, "w") as f:
    json.dump(params, f, indent=2)

# =========================
# SUMMARY
# =========================
print(f"[OK] ROI size: {w}x{h} @ ({x0},{y0})")
print(f"[OK] Template size: {TEMPLATE_W}x{TEMPLATE_H} (w x h)")
print(f"[OK] Score-map size: {max_score_map.shape[1]}x{max_score_map.shape[0]}")
print(f"[OK] Detections: {len(detections)} (metric={METRIC}, threshold={thr})")
print(f"[OUT] {boxes_path}, {heat_path}, {both_path}, {csv_path}, {params_path}")
if len(detections) > 0:
    print("Top 5 detections:", detections[:5])
else:
    print("No detections at current threshold. Consider lowering THRESH_ZNCC (e.g., 0.65) or adding another horizontal exemplar.")


[OK] ROI size: 7600x5740 @ (2112,2400)
[OK] Template size: 40x30 (w x h)
[OK] Score-map size: 7561x5711
[OK] Detections: 566 (metric=ZNCC, threshold=0.65)
[OUT] ./output-metal-via-contacts/mioc_roi_rect_H_overlay_boxes.png, ./output-metal-via-contacts/mioc_roi_rect_H_overlay_heatmap.png, ./output-metal-via-contacts/mioc_roi_rect_H_overlay_both.png, ./output-metal-via-contacts/mioc_roi_rect_H_detections.csv, ./output-metal-via-contacts/mioc_roi_rect_H_params.json
Top 5 detections: [{'x': 5849, 'y': 4227, 'score': 0.9999932646751404, 'template_id': 0, 'width': 40, 'height': 30}, {'x': 5935, 'y': 5382, 'score': 0.9999921917915344, 'template_id': 3, 'width': 40, 'height': 30}, {'x': 2153, 'y': 6837, 'score': 0.9999899864196777, 'template_id': 7, 'width': 40, 'height': 30}, {'x': 6179, 'y': 5425, 'score': 0.9999889731407166, 'template_id': 1, 'width': 40, 'height': 30}, {'x': 3701, 'y': 5597, 'score': 0.9999876618385315, 'template_id': 8, 'width': 40, 'height': 30}]


# STEP 3
#### -Combine both vertical and horizontal vias

In [4]:
# CHUNK 3: Merge V+H CSVs, dedupe, and produce combined overlays (red=vertical, green=horizontal).
# - Inputs: original image path, *_V_detections.csv, *_H_detections.csv, and the ROI used during detection.
# - Outputs: merged CSV, combined boxes overlay, combined heatmap, and both.
# - Adjust CONFIG paths/sizes/threshold before running.

import os
import csv
import json
from datetime import datetime

import numpy as np
import cv2

# =========================
# CONFIG (edit these)
# =========================
# IMAGE_PATH = "./images/ncr_8338d_mcmaster_mz_mit20x2.jpg"

CSV_V_PATH = "./output-metal-via-contacts/mioc_roi_rect_V_detections.csv"  # vertical detections (30x40)
CSV_H_PATH = "./output-metal-via-contacts/mioc_roi_rect_H_detections.csv"  # horizontal detections (40x30)

# If width/height columns are missing in CSVs, fall back to these defaults:
DEFAULT_V_W, DEFAULT_V_H = 30, 40   # vertical contact box (w x h)
DEFAULT_H_W, DEFAULT_H_H = 40, 30   # horizontal contact box (w x h)

# ROI used during the two detection runs (x0, y0, w, h)
# ROI = (2112, 2400, 7600, 5400)

# IoU threshold for merging duplicates (higher keeps fewer boxes)
IOU_THRESHOLD = 0.30

# Heatmap blend alpha and smoothing (kernel sizes must be odd)
HEATMAP_ALPHA = 0.42
HEAT_KX = 15  # suggested ~ (max width // 2) * 2 + 1
HEAT_KY = 21  # suggested ~ (max height // 2) * 2 + 1

# Output prefix (directory should exist)
OUTPUT_PREFIX = "./output-metal-via-contacts/mioc_roi_merge"

# =========================
# HELPERS
# =========================
def load_gray_image(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"Failed to read: {path}")
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def read_csv(path, default_w=None, default_h=None):
    rows = []
    with open(path, "r", newline="") as f:
        r = csv.DictReader(f)
        for d in r:
            try:
                x = int(float(d.get("x")))
                y = int(float(d.get("y")))
            except Exception:
                # Skip malformed
                continue
            score = float(d.get("score", 0.0))
            # width/height may be missing; use defaults if needed
            w = d.get("width")
            h = d.get("height")
            if w is None or h is None or str(w).strip()=="" or str(h).strip()=="":
                if default_w is None or default_h is None:
                    raise ValueError(f"{path}: missing width/height and no defaults provided.")
                w = default_w
                h = default_h
            else:
                w = int(float(w))
                h = int(float(h))
            tid = d.get("template_id")
            tid = int(tid) if tid is not None and str(tid).strip() != "" else -1
            rows.append({"x": x, "y": y, "score": score, "width": w, "height": h, "template_id": tid})
    return rows

def clip_box(tlx, tly, brx, bry, W, H):
    return max(0, tlx), max(0, tly), min(W-1, brx), min(H-1, bry)

def rect_from_center(x, y, w, h):
    half_w = w // 2
    half_h = h // 2
    tlx = x - half_w
    tly = y - half_h
    brx = tlx + w - 1
    bry = tly + h - 1
    return tlx, tly, brx, bry

def iou_rect(a, b):
    # a,b are (tlx,tly,brx,bry)
    tlx = max(a[0], b[0]); tly = max(a[1], b[1])
    brx = min(a[2], b[2]); bry = min(a[3], b[3])
    iw = brx - tlx + 1
    ih = bry - tly + 1
    if iw <= 0 or ih <= 0:
        return 0.0
    inter = iw * ih
    area_a = (a[2]-a[0]+1) * (a[3]-a[1]+1)
    area_b = (b[2]-b[0]+1) * (b[3]-b[1]+1)
    return inter / float(area_a + area_b - inter)

def nms_keep_best(dets, iou_thr, image_shape):
    """
    dets: list of dicts with x,y,score,width,height,orientation ('V' or 'H'), template_id
    Returns kept list after IoU-based dedupe (keep higher-score).
    """
    if not dets:
        return []
    H, W = image_shape
    # Precompute rects
    for d in dets:
        tlx, tly, brx, bry = rect_from_center(d["x"], d["y"], d["width"], d["height"])
        tlx, tly, brx, bry = clip_box(tlx, tly, brx, bry, W, H)
        d["_rect"] = (tlx, tly, brx, bry)

    # Sort by score desc
    dets_sorted = sorted(dets, key=lambda z: z["score"], reverse=True)
    kept = []
    for d in dets_sorted:
        discard = False
        for k in kept:
            if iou_rect(d["_rect"], k["_rect"]) >= iou_thr:
                discard = True
                break
        if not discard:
            kept.append(d)

    # Clean temp
    for d in kept:
        d.pop("_rect", None)
    return kept

def draw_combined_boxes(base_bgr, dets, color_V=(0,0,255), color_H=(0,255,0), thickness=2, labels=False):
    out = base_bgr.copy()
    H, W = out.shape[:2]
    for d in dets:
        x, y = d["x"], d["y"]
        w, h = d["width"], d["height"]
        tlx, tly, brx, bry = rect_from_center(x, y, w, h)
        tlx, tly, brx, bry = clip_box(tlx, tly, brx, bry, W, H)
        color = color_V if d["orientation"] == "V" else color_H
        cv2.rectangle(out, (tlx, tly), (brx, bry), color, thickness, lineType=cv2.LINE_AA)
        # Crosshair
        ch = max(3, min(w, h)//6)
        cv2.line(out, (x - ch, y), (x + ch, y), color, 1, lineType=cv2.LINE_AA)
        cv2.line(out, (x, y - ch), (x, y + ch), color, 1, lineType=cv2.LINE_AA)
        if labels:
            cv2.putText(out, f'{d["orientation"]}:{d["score"]:.2f}', (tlx, max(0, tly-2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1, cv2.LINE_AA)

    # Tiny legend
    legend = out.copy()
    cv2.rectangle(legend, (10,10), (220,60), (255,255,255), -1)
    cv2.putText(legend, "Legend:", (18,28), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(legend, "Vertical (30x40)", (18,46), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,0,255), 1, cv2.LINE_AA)
    cv2.putText(legend, "Horizontal (40x30)", (120,46), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,255,0), 1, cv2.LINE_AA)
    # Blend legend slightly for readability
    out = cv2.addWeighted(legend, 0.85, out, 0.15, 0)
    return out

def make_heatmap_overlay(base_gray_bgr, roi_rect, center_map, alpha=0.42):
    x0, y0, w, h = roi_rect
    cm = center_map.copy().astype(np.float32)
    if cm.max() > cm.min():
        cm = (cm - cm.min()) / (cm.max() - cm.min())
    cm_uint8 = np.clip(cm * 255.0, 0, 255).astype(np.uint8)
    color = cv2.applyColorMap(cm_uint8, cv2.COLORMAP_JET)
    overlay = base_gray_bgr.copy()
    roi_base = overlay[y0:y0+h, x0:x0+w]
    blended = cv2.addWeighted(color, alpha, roi_base, 1 - alpha, 0.0)
    overlay[y0:y0+h, x0:x0+w] = blended
    return overlay

# =========================
# LOAD DATA
# =========================
img_gray = load_gray_image(IMAGE_PATH)
H_full, W_full = img_gray.shape[:2]
img_bgr = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

V = read_csv(CSV_V_PATH, default_w=DEFAULT_V_W, default_h=DEFAULT_V_H)
H = read_csv(CSV_H_PATH, default_w=DEFAULT_H_W, default_h=DEFAULT_H_H)

for d in V: d["orientation"] = "V"
for d in H: d["orientation"] = "H"

# Restrict to ROI (safety) in case CSVs contain full-image detections
x0, y0, w, h = ROI
def in_roi(d):
    return (x0 <= d["x"] < x0+w) and (y0 <= d["y"] < y0+h)
V = [d for d in V if in_roi(d)]
H = [d for d in H if in_roi(d)]

all_dets = V + H

# =========================
# MERGE / DEDUPE (IoU NMS)
# =========================
kept = nms_keep_best(all_dets, IOU_THRESHOLD, image_shape=(H_full, W_full))

# Stats
nV = sum(1 for d in kept if d["orientation"]=="V")
nH = sum(1 for d in kept if d["orientation"]=="H")

# =========================
# HEATMAP (combined)
# =========================
# Place scores at centers within ROI; take max per pixel; smooth
center_map = np.zeros((h, w), dtype=np.float32)
for d in kept:
    cx = d["x"] - x0
    cy = d["y"] - y0
    if 0 <= cy < h and 0 <= cx < w:
        center_map[cy, cx] = max(center_map[cy, cx], d["score"])

# Ensure odd kernel sizes
kx = HEAT_KX if HEAT_KX % 2 == 1 else HEAT_KX + 1
ky = HEAT_KY if HEAT_KY % 2 == 1 else HEAT_KY + 1
center_map_blur = cv2.GaussianBlur(center_map, (kx, ky), 0)

overlay_heat = make_heatmap_overlay(img_bgr, ROI, center_map_blur, alpha=HEATMAP_ALPHA)

# =========================
# COMBINED BOXES & BOTH
# =========================
overlay_boxes = draw_combined_boxes(img_bgr, kept, color_V=(0,0,255), color_H=(0,255,0), thickness=2, labels=False)
overlay_both  = draw_combined_boxes(overlay_heat, kept, color_V=(0,0,255), color_H=(0,255,0), thickness=2, labels=False)

# =========================
# SAVE ARTIFACTS
# =========================
os.makedirs(os.path.dirname(OUTPUT_PREFIX), exist_ok=True)

csv_out   = f"{OUTPUT_PREFIX}_detections_merged.csv"
boxes_out = f"{OUTPUT_PREFIX}_overlay_boxes.png"
heat_out  = f"{OUTPUT_PREFIX}_overlay_heatmap.png"
both_out  = f"{OUTPUT_PREFIX}_overlay_both.png"
params_out = f"{OUTPUT_PREFIX}_params.json"

with open(csv_out, "w", newline="") as f:
    wcsv = csv.DictWriter(f, fieldnames=["x","y","score","orientation","width","height","template_id","band_id"])
    wcsv.writeheader()
    for d in kept:
        wcsv.writerow({
            "x": d["x"], "y": d["y"], "score": d["score"],
            "orientation": d["orientation"],
            "width": d["width"], "height": d["height"],
            "template_id": d.get("template_id", -1),
            "band_id": d.get("band_id", -1)
        })

cv2.imwrite(boxes_out, overlay_boxes)
cv2.imwrite(heat_out, overlay_heat)
cv2.imwrite(both_out, overlay_both)

params = {
    "timestamp": datetime.now().isoformat(),
    "image_path": os.path.abspath(IMAGE_PATH),
    "vertical_csv": os.path.abspath(CSV_V_PATH),
    "horizontal_csv": os.path.abspath(CSV_H_PATH),
    "roi": {"x0": x0, "y0": y0, "w": w, "h": h},
    "iou_threshold": IOU_THRESHOLD,
    "heatmap_alpha": HEATMAP_ALPHA,
    "heat_kernel": {"kx": kx, "ky": ky},
    "kept_counts": {"V": nV, "H": nH, "total": len(kept)},
    "notes": "Merged V/H with IoU-NMS (keep best by score). Colors: V=red, H=green."
}
with open(params_out, "w") as f:
    json.dump(params, f, indent=2)

print(f"[OK] Kept detections: total={len(kept)} | V={nV}, H={nH}")
print(f"[OUT] {csv_out}\n[OUT] {boxes_out}\n[OUT] {heat_out}\n[OUT] {both_out}\n[OUT] {params_out}")


[OK] Kept detections: total=975 | V=409, H=566
[OUT] ./output-metal-via-contacts/mioc_roi_merge_detections_merged.csv
[OUT] ./output-metal-via-contacts/mioc_roi_merge_overlay_boxes.png
[OUT] ./output-metal-via-contacts/mioc_roi_merge_overlay_heatmap.png
[OUT] ./output-metal-via-contacts/mioc_roi_merge_overlay_both.png
[OUT] ./output-metal-via-contacts/mioc_roi_merge_params.json


# STEP 4
#### -Iterative step to merge int missing contacts, and remove extra contacts

In [5]:
IMG_PATH = "./images/ncr_8338d_mcmaster_mz_mit20x2.jpg"       
ROI = (2112, 2400, 7600, 5740)  # <-- EDIT to your test region
SEED_PATH = "./mioc_annotated_netlist.txt" 
OUTPUT_DIR = "./output-metal-via-contacts"

In [6]:
CONTACTS_ADD   = [(3960, 4060, "V"), (3960,5051,"V"), (8070,4640,"V")]
CONTACTS_REMOVE = [(3960,3926)] # (1348,1598)]

In [7]:
# === Stage 1 (final) — detections -> GENERATED, optional add/remove, FULL-FRAME overlay ===
# Set these globals before running (example values shown):
# IMG_PATH   = "./images/ncr_8338d_mcmaster_mz_mit20x2.jpg"
# ROI        = (2112, 2400, 7600, 5740)   # or None; used ONLY to FILTER detections
# SEED_PATH  = "./mioc_annotated_netlist.txt"
# OUTPUT_DIR = "./output-metal-via-contacts"
#
# Optional user edits (can be empty or omitted):
# CONTACTS_ADD    = [(x, y, "H"|"V"[, "label"]), ...]
# CONTACTS_REMOVE = [(x, y), ...]   # removes the SINGLE nearest detected contact for each point
#
# Outputs (will OVERWRITE if they already exist):
#   If no add/remove given:
#     - ./<seed>_with_generated.txt
#     - <OUTPUT_DIR>/<seed>_with_generated.txt
#     - <OUTPUT_DIR>/overlay_seed_generated_contacts.png
#   If any add/remove given:
#     - ./<seed>_with_generated_corrected.txt
#     - <OUTPUT_DIR>/<seed>_with_generated_corrected.txt
#     - <OUTPUT_DIR>/overlay_seed_generated_contacts_corrected.png
#
# Colors: Generated VPIN=RED, HPIN=GREEN; SEED RPIN=PURPLE

from pathlib import Path
import re, os, math
import pandas as pd
import numpy as np
import cv2

# -----------------------
# Tunables
# -----------------------
ADD_DEDUP_TOL = 2  # px: skip additions if within this L-inf distance of an existing candidate

# -----------------------
# Helpers
# -----------------------
def find_section(lines, name):
    start_idx = end_idx = None
    for i, ln in enumerate(lines):
        if re.match(rf"^\s*SECTION\s*,\s*{re.escape(name)}\s*,\s*START\s*$", ln, re.I):
            start_idx = i
        if re.match(rf"^\s*SECTION\s*,\s*{re.escape(name)}\s*,\s*END\s*$", ln, re.I):
            end_idx = i
            break
    return (start_idx, end_idx) if (start_idx is not None and end_idx is not None and end_idx > start_idx) else None

def remove_generated_block(lines):
    sec = find_section(lines, "GENERATED")
    if not sec: return lines, False
    s,e = sec
    return lines[:s] + lines[e+1:], True

def insert_generated_block(lines, generated_rows):
    block = ["SECTION, GENERATED, START"] + generated_rows + ["SECTION, GENERATED, END"]
    insert_idx = len(lines)
    for key in ("DEFINITIONS", "NOTES"):
        sec = find_section(lines, key)
        if sec: insert_idx = min(insert_idx, sec[0])
    return lines[:insert_idx] + block + lines[insert_idx:]

def load_device_sizes(text):
    sizes = {}
    for ln in text.splitlines():
        if re.match(r"^\s*DEVICE\s*,", ln, re.I):
            parts = [p.strip() for p in ln.split(",")]
            if len(parts) >= 4:
                _, name, w, h, *rest = parts
                try: sizes[name] = (int(float(w)), int(float(h)))
                except: pass
    sizes.setdefault("HPIN", (40,30))
    sizes.setdefault("VPIN", (30,40))
    sizes.setdefault("PAD",  (520,520))
    return sizes

def parse_seed_rpins(text):
    """Collect RPINs from SECTION, SEED (for purple overlay)."""
    lines = text.splitlines()
    sec = find_section(lines, "SEED")
    rpins = []
    if not sec: return rpins
    s,e = sec
    for ln in lines[s+1:e]:
        if re.match(r"^\s*RPIN\s*,", ln, re.I):
            parts = [p.strip() for p in ln.split(",")]
            # RPIN, <device>, <id>, <x>, <y>, <N>, <S>, <E>, <W>[, <label>]
            if len(parts) >= 9:
                _, dev, pid, xs, ys, *_ = parts
                try:
                    x = int(float(xs)); y = int(float(ys))
                except:
                    continue
                rpins.append({"id": pid, "device": dev, "x": x, "y": y})
    return rpins

def col(df, names, default=None):
    for n in names:
        if n in df.columns: return df[n]
    if default is None: return pd.Series([None]*len(df))
    return pd.Series([default]*len(df))

def infer_device_from_orientation(ori):
    if isinstance(ori, str):
        o = ori.strip().upper()
        if o in ("H","HORZ","HORIZONTAL"): return "HPIN"
        if o in ("V","VERT","VERTICAL"):   return "VPIN"
    return None

def draw_rect(img, cx, cy, w, h, color, thick=2):
    x0 = int(round(cx - w/2)); y0 = int(round(cy - h/2))
    x1 = int(round(cx + w/2)); y1 = int(round(cy + h/2))
    cv2.rectangle(img, (x0,y0), (x1,y1), color, thick)

def normalize_roi(roi, img_shape):
    if roi is None: return None
    if len(roi) != 4: raise ValueError("ROI must be None or a 4-tuple/list")
    H, W = img_shape[:2]
    x, y, a, b = map(int, roi)
    if a >= 0 and b >= 0:  # (x,y,w,h)
        x0, y0, x1, y1 = x, y, x + a, y + b
    else:                  # (x0,y0,x1,y1)
        x0, y0, x1, y1 = x, y, a, b
    x0, y0 = max(0,x0), max(0,y0)
    x1, y1 = min(W,x1), min(H,y1)
    if x1 <= x0 or y1 <= y0: raise ValueError("ROI is empty after clamping")
    return (x0, y0, x1, y1)

def euclid2(a, b):
    dx = a[0]-b[0]; dy = a[1]-b[1]
    return dx*dx + dy*dy

# -----------------------
# Resolve paths & load inputs
# -----------------------
OUTPUT_DIR = Path(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

img_path  = Path(IMG_PATH)
seed_path = Path(SEED_PATH)
det_path  = OUTPUT_DIR / "mioc_roi_merge_detections_merged.csv"
if not det_path.exists():
    alt = Path("mioc_roi_merge_detections_merged.csv")
    if alt.exists(): det_path = alt
    else: raise FileNotFoundError("mioc_roi_merge_detections_merged.csv not found in OUTPUT_DIR or cwd")

# Image (FULL-FRAME overlay; we never crop)
img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError(f"Could not read image at {img_path}")
img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
roi_box = normalize_roi(ROI, img_color.shape) if ROI is not None else None

# SEED & device sizes
seed_text    = seed_path.read_text(encoding="utf-8", errors="ignore")
device_sizes = load_device_sizes(seed_text)
seed_rpins   = parse_seed_rpins(seed_text)

# Detections
df = pd.read_csv(det_path)
xs   = col(df, ["x","X","cx","center_x"]).astype(float)
ys   = col(df, ["y","Y","cy","center_y"]).astype(float)
oris = col(df, ["orientation","ori","orient"])
ws   = col(df, ["width","w"]).astype(float)
hs   = col(df, ["height","h"]).astype(float)
labs = col(df, ["label","cpin","name","rpin"], default="").astype(str)

# ROI filter (FILTER ONLY; no coordinate shift)
if roi_box:
    x0,y0,x1,y1 = roi_box
    keep = (xs>=x0)&(xs<x1)&(ys>=y0)&(ys<y1)
    xs, ys, oris, ws, hs, labs = xs[keep], ys[keep], oris[keep], ws[keep], hs[keep], labs[keep]
    df = df[keep].copy()

# Build candidates from detections
candidates = []
for i, (x,y,o) in enumerate(zip(xs, ys, oris), start=1):
    dev = infer_device_from_orientation(o)
    if dev is None:
        wv = ws.iloc[i-1] if len(ws)>=i else np.nan
        hv = hs.iloc[i-1] if len(hs)>=i else np.nan
        if pd.notna(wv) and pd.notna(hv):
            dev = "HPIN" if float(wv) >= float(hv) else "VPIN"
        else:
            dev = "VPIN"
    lab = labs.iloc[i-1] if len(labs)>=i else ""
    candidates.append({"x": int(round(x)), "y": int(round(y)), "dev": dev, "label": (lab or "").strip()})

# -----------------------
# Optional REMOVALS (single nearest)
# -----------------------
try:
    RM_LIST = CONTACTS_REMOVE
    if not isinstance(RM_LIST, (list, tuple)): RM_LIST = []
except NameError:
    RM_LIST = []

removed = []
for (rx, ry) in RM_LIST:
    try:
        rx_i, ry_i = int(round(float(rx))), int(round(float(ry)))
    except:
        continue
    if not candidates: break
    best_k, best_d2 = -1, float("inf")
    for k, c in enumerate(candidates):
        d2 = euclid2((rx_i, ry_i), (c["x"], c["y"]))
        if d2 < best_d2:
            best_d2, best_k = d2, k
    if best_k >= 0:
        removed.append(candidates[best_k])
        del candidates[best_k]

# -----------------------
# Optional ADDITIONS (skip near-dupes)
# -----------------------
try:
    ADD_LIST = CONTACTS_ADD
    if not isinstance(ADD_LIST, (list, tuple)): ADD_LIST = []
except NameError:
    ADD_LIST = []

def is_near_existing(x, y, pts, tol=ADD_DEDUP_TOL):
    for p in pts:
        if abs(p["x"] - x) <= tol and abs(p["y"] - y) <= tol:
            return True
    return False

added = []
for t in ADD_LIST:
    if len(t) < 3: continue
    x, y, ori = t[0], t[1], t[2]
    lbl = t[3] if len(t) >= 4 else None
    try:
        x_i, y_i = int(round(float(x))), int(round(float(y)))
    except:
        continue
    # (Optionally require ROI membership—currently we allow anywhere since overlay is full-frame)
    dev = infer_device_from_orientation(ori)
    if dev is None:
        s = str(ori).strip().lower()
        if s.startswith("h"): dev = "HPIN"
        elif s.startswith("v"): dev = "VPIN"
    if dev not in ("HPIN","VPIN"): continue
    if is_near_existing(x_i, y_i, candidates): continue
    rec = {"x": x_i, "y": y_i, "dev": dev, "label": (str(lbl).strip() if lbl is not None else "")}
    candidates.append(rec)
    added.append(rec)

# -----------------------
# Generate fresh GENERATED section
# -----------------------
generated_rows = []
h_count = v_count = 0
for idx, c in enumerate(candidates, start=1):
    pid = f"RGEN_{idx:05d}"
    core = f"RPIN, {c['dev']}, {pid}, {c['x']}, {c['y']}, NULL, NULL, NULL, NULL"
    line = f"{core}, {c['label']}" if c['label'] else core
    generated_rows.append(line)
    if c['dev'] == "HPIN": h_count += 1
    elif c['dev'] == "VPIN": v_count += 1

# Prepare annotated outputs (no incremental numbering; always overwrite)
seed_lines = seed_text.splitlines()
seed_lines, _ = remove_generated_block(seed_lines)
out_lines = insert_generated_block(seed_lines, generated_rows)

seed_stem = seed_path.stem
made_corrections = (len(removed) > 0) or (len(added) > 0)
annot_name = f"{seed_stem}_with_generated_corrected.txt" if made_corrections else f"{seed_stem}_with_generated.txt"
overlay_name = "overlay_seed_generated_contacts_corrected.png" if made_corrections else "overlay_seed_generated_contacts.png"

annot_path_cwd = Path.cwd() / annot_name
annot_path_out = OUTPUT_DIR / annot_name
overlay_path   = OUTPUT_DIR / overlay_name

# Write annotated (overwrite)
annot_text = "\n".join(out_lines) + "\n"
annot_path_cwd.write_text(annot_text, encoding="utf-8")
annot_path_out.write_text(annot_text, encoding="utf-8")

# -----------------------
# Draw overlay (FULL-FRAME, NO LEGEND, NO CROPPING)
# -----------------------
overlay = img_color.copy()
w_hpin, h_hpin = device_sizes.get("HPIN", (40,30))
w_vpin, h_vpin = device_sizes.get("VPIN", (30,40))

# Generated
for c in candidates:
    if c["dev"] == "HPIN":
        draw_rect(overlay, c["x"], c["y"], w_hpin, h_hpin, (0,255,0), 2)  # GREEN
    else:
        draw_rect(overlay, c["x"], c["y"], w_vpin, h_vpin, (0,0,255), 2)  # RED

# SEED RPINs
def size_for_device(dev):
    return device_sizes.get(dev, device_sizes.get("HPIN", (40,30)))
for rp in seed_rpins:
    w,h = size_for_device(rp["device"])
    draw_rect(overlay, rp["x"], rp["y"], w, h, (255,0,255), 2)            # PURPLE

# Save FULL-FRAME overlay (overwrite)
cv2.imwrite(str(overlay_path), overlay)

# -----------------------
# Report
# -----------------------
print("=== Stage 1 (final) complete ===")
print(f"Image:                  {img_path}")
print(f"Detections CSV:         {det_path}")
print(f"Seed in:                {seed_path}")
print(f"Annotated (cwd):        {annot_path_cwd}")
print(f"Annotated (output dir): {annot_path_out}")
print(f"Overlay:                {overlay_path}")
if roi_box: print(f"ROI filter used:        (x0,y0,x1,y1) = {roi_box}")
print(f"Generated totals:       HPIN={h_count}, VPIN={v_count}, TOTAL={len(candidates)}")
print(f"Removals requested:     {len(RM_LIST)}  -> removed {len(removed)}")
print(f"Additions requested:    {len(ADD_LIST)} -> added {len(added)} (dedup tol {ADD_DEDUP_TOL}px)")


=== Stage 1 (final) complete ===
Image:                  images/ncr_8338d_mcmaster_mz_mit20x2.jpg
Detections CSV:         output-metal-via-contacts/mioc_roi_merge_detections_merged.csv
Seed in:                mioc_annotated_netlist.txt
Annotated (cwd):        /build/ws/ediaz/repo/jupyter/mioc/mioc_annotated_netlist_with_generated_corrected.txt
Annotated (output dir): output-metal-via-contacts/mioc_annotated_netlist_with_generated_corrected.txt
Overlay:                output-metal-via-contacts/overlay_seed_generated_contacts_corrected.png
ROI filter used:        (x0,y0,x1,y1) = (2112, 2400, 9712, 8140)
Generated totals:       HPIN=565, VPIN=412, TOTAL=977
Removals requested:     1  -> removed 1
Additions requested:    3 -> added 3 (dedup tol 2px)
